In [ ]:
#%pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import datetime
import mediapipe as mp

In [ ]:
# Diretório para os pontos coletados
DATA_DIR = os.path.join('Dados_extraidos') 

# Sinais
sinais = np.array(['Ola','Tudo bem','Obrigado','Entendo','Calma'])

# Quantidade de vídeos por sinal
num_videos = 10

# Quantidade de frames por vídeo
num_frames = 30

In [ ]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
cam = cv2.VideoCapture(0)

In [ ]:
def mediapipe_extrai(imagem, modelo):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB) 
    imagem.flags.writeable = False                  
    resultados = modelo.process(imagem)                
    imagem.flags.writeable = True                   
    imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2BGR) 
    return imagem, resultados

In [ ]:
def mediapipe_desenha(imagem, resultados):
    # Pontos do rosto
    mp_drawing.draw_landmarks(imagem, resultados.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(153,0,153), thickness=1, circle_radius=1), 
                             mp_drawing_styles.get_default_face_mesh_contours_style()
                             ) 
    # Pontos do corpo
    mp_drawing.draw_landmarks(imagem, resultados.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(153,0,153), thickness=2, circle_radius=3), 
                             mp_drawing.DrawingSpec(color=(255,40,130), thickness=2, circle_radius=1)
                             ) 
    # Pontos da mão esquerda
    mp_drawing.draw_landmarks(imagem, resultados.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(153,0,153), thickness=1, circle_radius=3), 
                             mp_drawing.DrawingSpec(color=(35,100,255), thickness=1, circle_radius=1)
                             ) 
    # Pontos da mão direita  
    mp_drawing.draw_landmarks(imagem, resultados.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(153,0,153), thickness=1, circle_radius=3), 
                             mp_drawing.DrawingSpec(color=(247,0,247), thickness=1, circle_radius=1)
                             ) 

In [ ]:
def mediapipe_desenha_extrai(cam, modelo):
    ret, frame = cam.read()                             # Ler o frame
    imagem, resultados = mediapipe_extrai(frame, modelo)# Fazer detecções
    mediapipe_desenha(imagem, resultados)                # Desenhar pontos
    return imagem, resultados

In [ ]:
def trata_pontos(resultados):

    rosto = []
    corpo = []
    mao_esquerda = []
    mao_direita = []

    if resultados.face_landmarks:
        for r in resultados.face_landmarks.landmark:
            ponto = np.array([r.x, r.y, r.z])
            rosto.append(ponto)
    else:
        rosto = np.zeros(468*3) 

    if resultados.pose_landmarks:
        for r in resultados.pose_landmarks.landmark:
            ponto = np.array([r.x, r.y, r.z, r.visibility])
            corpo.append(ponto)
    else:
        corpo = np.zeros(33*4) 
    
    if resultados.left_hand_landmarks:
        for r in resultados.left_hand_landmarks.landmark:
            ponto = np.array([r.x, r.y, r.z])
            mao_esquerda.append(ponto)
    else:
        mao_esquerda = np.zeros(21*3)   

    if resultados.right_hand_landmarks:
        for r in resultados.right_hand_landmarks.landmark:
            ponto = np.array([r.x, r.y, r.z])
            mao_direita.append(ponto)
    else:
        mao_direita = np.zeros(21*3)

    rosto = np.array(rosto).flatten() 
    corpo = np.array(corpo).flatten()
    mao_esquerda = np.array(mao_esquerda).flatten()
    mao_direita = np.array(mao_direita).flatten()

    return np.concatenate([corpo, rosto, mao_esquerda, mao_direita])    

In [ ]:
# Monta toda estrutura de diretórios
for sinal in sinais: 
    for video in range(num_videos):
        try: 
            os.makedirs(os.path.join(DATA_DIR, sinal, str(video)))
        except:
            pass

In [ ]:
with mp_holistic.Holistic() as holistic:
    
    while cam.isOpened():

        # Desenha pontos na imagem e extrai características em resultados
        imagem, resultados = mediapipe_desenha_extrai(cam, holistic)

        # Enunciado na tela
        cv2.putText(imagem,'Por favor, posicione-se da cintura para cima',(20, 40),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.8,color=(0, 255, 255), thickness = 2,lineType=cv2.LINE_AA)  
        cv2.putText(imagem,'Tecle barra para continuar',(20, 80),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 255, 255), thickness = 2,lineType=cv2.LINE_AA)  
           
        # Mostra a tela
        cv2.imshow('LIBRAS', imagem)

        # Espera pela tecla
        if cv2.waitKey(10) & 0xFF == ord(' '):
            cv2.destroyAllWindows()
            break


In [ ]:
x_polegar_direito = resultados.pose_landmarks.landmark[22].x
pontos_tratados = trata_pontos(resultados)
indice = np.where(pontos_tratados == x_polegar_direito)[0]
print(indice)

In [ ]:
with mp_holistic.Holistic() as holistic:

    for sinal in sinais:

        #video inicial
        video = 0

        while video < num_videos:
          
            # Enunciado
            while cam.isOpened():
                imagem, resultados = mediapipe_desenha_extrai(cam, holistic)
                cv2.putText(imagem,'Sinal {}, video numero {}'.format(sinal, video),(20, 40),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 255, 255), thickness = 2)  
                cv2.putText(imagem,'Tecle barra para gravar',(20, 80),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 255, 255), thickness = 2)  
                cv2.imshow('LIBRAS', imagem)
                if cv2.waitKey(10) & 0xFF == ord(' '):
                    break

            # Contagem regressiva
            contador = 3
            inicio = time.time()
            while contador > 0:
                imagem, resultados = mediapipe_desenha_extrai(cam, holistic)
                cv2.putText(imagem,'Preparando pra gravar: {}'.format(contador),(20, 40),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 255, 255), thickness = 2)
                cv2.imshow('LIBRAS', imagem)
                cv2.waitKey(10)
                final = time.time()
                if final - inicio >= 1:
                    contador-=1
                    inicio=final

            # Gravando
            for frame_num in range(num_frames):

                # Desenha pontos na imagem e extrai características em resultados
                imagem, resultados = mediapipe_desenha_extrai(cam, holistic)

                # Mostra na tela
                cv2.putText(imagem,'Sinal {}, video numero {}'.format(sinal, video),(20, 40),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 255, 255), thickness = 2)  
                cv2.putText(imagem,'Gravando...',(20, 80),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 0, 255), thickness = 2)  
                cv2.imshow('LIBRAS', imagem)
                cv2.waitKey(10)

                # Trata os pontos e arquiva
                pontos_tratados = trata_pontos(resultados)
                nome_arquivo = os.path.join(DATA_DIR, sinal, str(video), str(frame_num))
                np.save(nome_arquivo, pontos_tratados)

            # Gravar novamente
            while cam.isOpened():
                # Desenha pontos na imagem e extrai características em resultados
                imagem, resultados = mediapipe_desenha_extrai(cam, holistic)
                cv2.putText(imagem,'Sinal {}, video numero {}'.format(sinal, video),(20, 40),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 255, 255), thickness = 2)  
                cv2.putText(imagem,'Para repetir, tecle "r"',(20, 80),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 255, 255), thickness = 2)
                cv2.putText(imagem,'Para prosseguir, tecle barra',(20, 120),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 255, 255), thickness = 2)    
                cv2.imshow('LIBRAS', imagem)
                k = cv2.waitKey(10)
                if k & 0xFF == ord('r'):
                    video-=1
                    break
                if k & 0xFF == ord(' '):
                    break
            video +=1

cam.release()
cv2.destroyAllWindows()

In [ ]:
# Zipando a pasta dos dados coletados
import shutil
shutil.make_archive('Dados_extraidos', 'zip', 'Dados_extraidos')